In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import matplotlib.colors as mcolors

def setup_mpl():
    mpl.rc('font', size = 10)
    mpl.rcParams['legend.fontsize'] = 'small'
    mpl.rcParams['xtick.labelsize'] = 'small'
    mpl.rcParams['ytick.labelsize'] = 'small'
    #
    mpl.rcParams['font.family'] = 'Helvetica'
    mpl.rcParams['mathtext.default'] = 'regular'
    #
    mpl.rcParams['lines.linewidth'] = 1
    mpl.rcParams['lines.markersize'] = 6  
    mpl.rcParams['axes.linewidth'] = 0.75
    mpl.rcParams['axes.labelpad'] = 2
    #
    mpl.rcParams['xtick.major.pad'] = '2.3'
    mpl.rcParams['ytick.major.pad'] = '2.3'
    #
    #
    mpl.rcParams['xtick.major.width'] = 0.75
    mpl.rcParams['ytick.major.width'] = 0.75
    mpl.rcParams['xtick.minor.width'] = 0.75
    mpl.rcParams['ytick.minor.width'] = 0.75
    #
    mpl.rcParams['xtick.major.size'] = 3
    mpl.rcParams['ytick.major.size'] = 3
    #
    mpl.rcParams['xtick.minor.size'] = 1.5
    mpl.rcParams['ytick.minor.size'] = 1.5
    #
    alpha = 0.6
    to_rgba = mpl.colors.ColorConverter().to_rgba
setup_mpl()

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import geopandas as gpd
import hdbscan
from shapely.geometry import Point
import ast
import radialtree as rt
import scipy.cluster.hierarchy as sch
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score
import numpy as np
import pickle

In [3]:
df_kommune = gpd.read_file("../data/kommunes.geojson")
kommune_stores = gpd.read_file("../data/stores_location.geojson")

In [ ]:
region_colors = {
    "Hovedstaden": "firebrick",
    "Sjælland": "lightcoral",
    "Syddanmark": "lightseagreen",
    "Midtjylland": "skyblue",
    "Nordjylland": "dodgerblue"
}
df_kommune["color"] = df_kommune["NAME_1"].map(region_colors)

In [ ]:
fig = plt.figure(figsize=(6,4),dpi = 300)
ax = fig.add_subplot(111)
df_kommune.plot(ax = ax,column="NAME_1", edgecolor = "black", linewidth = 0.5, legend = False, color = df_kommune["color"])
kommune_stores.plot(ax = ax, color = "#FFD700",markersize = 1.5)
ax.set_xlim([8,13])
fig.savefig("../figure/fig_1_denmark.pdf",bbox_inches = "tight",dpi = 300)

In [ ]:
G = nx.read_gml('data/stores/distance_stores.gml')
nG = nx.Graph()
stores_ids = kommune_stores["store_id"].tolist()
for (u,v,d) in G.edges(data = True):
    if u not in stores_ids or v not in stores_ids: continue
    nG.add_edge(u,v,weight = d['distance'])

In [ ]:
mapping_id_label = {i:label for i,label in enumerate(nG.nodes())}
mapping_region = {row['store_id']:row["NAME_1"] for i,row in kommune_stores.iterrows()}
mapping_store_id_region = [mapping_region[mapping_id_label[i]] for i in range(len(mapping_id_label))]
matrix_distance = nx.to_scipy_sparse_array(nG)

In [ ]:
matrix_distance = nx.to_scipy_sparse_array(nG)
clustering = hdbscan.HDBSCAN(metric='precomputed', min_cluster_size = 10)
clustering.fit(matrix_distance)

In [ ]:
colors_region = []
for j in mapping_store_id_region:
    colors_region.append(region_colors[j])
colors = {"Region": colors_region,}

In [ ]:
def silver_link_color_func(link_id):
    return '#C0C0C0'
linkage_tree = clustering.single_linkage_tree_.to_numpy()
Z2 = sch.dendrogram(linkage_tree,labels = ["" for i in range(len(stores_ids))],no_plot=True,color_threshold=0,above_threshold_color='C0')
fig = plt.figure(figsize=(7,7),dpi = 300)
ax = fig.add_subplot(1,1,1)
rt.radialTreee(Z2,ax = ax,colorlabels = colors)
fig.savefig("../figure/fig_1_hdbscan.pdf",bbox_inches = "tight",dpi = 300)

In [ ]:
labels = clustering.labels_
noise_mask = (labels == -1)
cluster_mask = ~noise_mask
gdf_projected = kommune_stores.to_crs(epsg=32633)
points_metric = np.array([[point.x, point.y] for point in gdf_projected.geometry])
cluster_points = points_metric[cluster_mask]
cluster_labels = labels[cluster_mask]
db_score = davies_bouldin_score(cluster_points, cluster_labels)
ch_score = calinski_harabasz_score(cluster_points, cluster_labels)
silhouette = silhouette_score(cluster_points, cluster_labels)
max_persistence = np.max(clustering.cluster_persistence_)
print(f"Davies-Bouldin Index: {round(db_score,2)}")
print(f"Calinski-Harabasz Index: {round(ch_score,0)}")
print(f"Silhouette Score: {round(silhouette,2)}")
print(f"Max Persistence: {round(max_persistence,3)}")

In [ ]:
hdbscan_stores = pd.read_file("../data/entropy_stores.csv")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=300)
sns.boxplot(ax = ax, data = hdbscan_stores, y = "entropy",hue = "region", fill = True,linewidth=0.75,palette = region_colors, showfliers = False,legend = False)
ax.set_ylabel("Entropy")
ax.set_ylim([0.7, 0.9])
yticks = np.arange(0.7, 0.91, 0.05)
ax.set_yticks(yticks)
ax.set_yticklabels([f"{i:.2f}" for i in yticks])
fig.tight_layout()
fig.savefig("../figure/fig_1_entropy.pdf",bbox_inches = "tight",dpi = 300)

In [ ]:
giant_component_df = pd.read_csv("../data/giant_component.csv")
mantel_df = pd.read_csv("../data/mantel_correlogram.csv")

In [ ]:
fig = plt.figure(figsize=(9.5,2),dpi = 300)
ax = fig.add_subplot(1,1,1)
tax = ax.twinx()
ax.plot(giant_component_df["distance"],giant_component_df["second_largest_cc"]/giant_component_df["second_largest_cc"].max(),color = "#ffa246",linewidth = 1.5,ls = "dotted",label = "Second Largest Cluster Size")
ax.plot(giant_component_df["distance"],giant_component_df["largest_cc"]/giant_component_df["largest_cc"].max(),color = "#2637BA",linewidth = 2,label = "First Largest Cluster Size")
tax.plot(mantel_df["bin_centers"],mantel_df["mantel_distance"],color = "#2DB469",linewidth = 2,label = "Mantel Correlation")
fdf = mantel_df[mantel_df["p_value_distance"] > 0.05]
fdf = fdf[fdf["bin_centers"] > 3000]
tax.fill_between(fdf["bin_centers"], fdf["mantel_distance"]+0.025, fdf["mantel_distance"]-0.025, color="#2DB469", alpha=0.2)
#ax.set_ylim([0,600])
#ax.set_yticks([0,200,400,600])
ax.set_xlim([1_000,100_000])
tax.set_ylim([-0.15,0.15])
tax.set_yticks([-0.15,-0.1,-0.05,0,0.05,0.1,0.15])
ax.set_xscale("log")
ax.set_ylabel("Normalized Size")
ax.set_xlabel("Distance [m]")
tax.set_ylabel("Mantel Correlation",rotation = 270,labelpad = 10)
ax.axvline(26_500,color = "silver",linestyle = "--",lw = 1)
tax.axhline(0.0,color = "silver",linestyle = "--",lw = 1)
ax.axvline(9_500,color = "silver",linestyle = "--",lw = 1)
ax.axvline(3750,color = "silver",linestyle = "--",lw = 1)
handles_ax, labels_ax = ax.get_legend_handles_labels()
handles_tax, labels_tax = tax.get_legend_handles_labels()
all_handles = handles_ax + handles_tax
all_labels = labels_ax + labels_tax
ax.legend(handles=all_handles, labels=all_labels, 
          bbox_to_anchor=[0.5, 1.1], 
          loc='center', 
          ncol=3, # Adjusted ncol to fit the 3 labels
          frameon=False)
fig.savefig("../figure/fig_1_mantel.pdf",bbox_inches = "tight",dpi = 300)

In [ ]:
distances = [3750,9_500,26_500]
dict_plot_distances = {k:[[],[]] for k in distances}
distances_df = pd.read_csv("data/stores/hdbscan_distances.csv",sep = ",")
store_location = gpd.read_file("../data/stores_location.geojson")
store_loc = {row["store_id"]:row["geometry"] for _,row in store_location.iterrows()}
M = distances_df.values
for distance in distances:
    #
    condition = (M > 0) & (M <= distance)
    G = nx.Graph()
    rows, cols = np.where(condition)
    G.add_edges_from(zip(rows, cols))
    #
    components = list(nx.connected_components(G))
    sorted_components = sorted(components, key=len, reverse=True)
    LCC = sorted_components[0]
    for i in LCC:
        dict_plot_distances[distance][0].append(store_loc[mapping_id_label[i]])
    second_largest_cc_size = len(sorted_components[1]) if len(sorted_components) > 1 else 0
    if second_largest_cc_size > 0:
        SCC = sorted_components[1]
        for i in SCC:
            dict_plot_distances[distance][1].append(store_loc[mapping_id_label[i]])

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(9.5, 3), dpi=300, 
                                     subplot_kw={'aspect': 'equal'})
df_kommune[df_kommune["NAME_1"].isin(["Hovedstaden"])].plot(
    ax=ax1, column="NAME_1", edgecolor="black", linewidth=0.25, legend=False, 
    color=df_kommune[df_kommune["NAME_1"].isin(["Hovedstaden"])]["color"], alpha=1
)
d = distances[0]
stores = dict_plot_distances[d][0]
X = np.array([ast.literal_eval(s) for s in stores])
gdf = gpd.GeoDataFrame(geometry=[Point(xy) for xy in X], crs='EPSG:4326')
gdf.plot(ax=ax1, color="#2637BA", markersize=10)
ax1.set_xlim([12.3, 12.7])
ax1.set_ylim([55.6, 55.8])
df_kommune[df_kommune["NAME_1"].isin(["Hovedstaden", "Sjælland"])].plot(
    ax=ax2, column="NAME_1", edgecolor="black", linewidth=0.25, legend=False, 
    color=df_kommune[df_kommune["NAME_1"].isin(["Hovedstaden", "Sjælland"])]["color"], alpha=1
)
d = distances[1]
stores = dict_plot_distances[d][0]
X = np.array([ast.literal_eval(s) for s in stores])
gdf = gpd.GeoDataFrame(geometry=[Point(xy) for xy in X], crs='EPSG:4326')
gdf.plot(ax=ax2, color="#2637BA", markersize=5)
ax2.set_xlim([12, 12.7])
ax2.set_ylim([55.5, 55.9])
df_kommune.plot(ax=ax3, column="NAME_1", edgecolor="black", linewidth=0.25, legend=False, color=df_kommune["color"],alpha = 1)
d = distances[-1]
stores_1 = dict_plot_distances[d][0]
X1 = np.array([ast.literal_eval(s) for s in stores_1])
gdf1 = gpd.GeoDataFrame(geometry=[Point(xy) for xy in X1], crs='EPSG:4326')
gdf1.plot(ax=ax3, color="#2637BA", markersize=1)
stores_2 = dict_plot_distances[d][1]
X2 = np.array([ast.literal_eval(s) for s in stores_2])
gdf2 = gpd.GeoDataFrame(geometry=[Point(xy) for xy in X2], crs='EPSG:4326')
gdf2.plot(ax=ax3, color="#ffa246", markersize=1)
ax3.set_xlim([8, 12.7])
ax3.set_ylim([54.5, 58])
fig.tight_layout()
fig.savefig("../figure/fig_1_blobs.pdf",bbox_inches = "tight",dpi = 300)

In [ ]:
all_vals = pd.read_csv("../data/distribution_stores.csv")
with open('../data/distribution_stores.pkl', 'rb') as f:
    region_lines = pickle.load(f)
max_by_cat = all_vals.max(axis=1)
ordered_categories = max_by_cat.sort_values(ascending=False).index.tolist()
fig = plt.figure(figsize=(3, 4), dpi=300)
ax = fig.add_subplot(1, 1, 1)
n_regions = 5
region_offset = 0.12
category_spacing = 1.0
for i, (region, (cats, vals)) in enumerate(region_lines.items()):
    vals_ord = np.array([vals[list(cats).index(cat)] for cat in ordered_categories])
    offset = (i - (n_regions-1)/2) * region_offset
    y_positions = np.arange(len(ordered_categories)) * category_spacing + offset
    ss = []
    for y, x in zip(y_positions, vals_ord):
        if x < 0.05:
            s = 12.5
            ax.hlines(y, 0, x, color=region_colors[region], alpha=0.7, linewidth=1)
        else:
            s = 25
            ax.hlines(y, 0, x, color=region_colors[region], alpha=0.7, linewidth=2)
        ss.append(s)
    ax.scatter(vals_ord, y_positions, color=region_colors[region], label=region, s=ss)
ax.set_yticks(np.arange(len(ordered_categories)) * category_spacing, ordered_categories)
ax.set_xlabel("Frequency")
ax.set_xlim(0, 0.22)
max_y = (len(ordered_categories) - 1) * category_spacing
max_offset = (n_regions-1)/2 * region_offset+0.25
ax.set_ylim(-max_offset - 0.1, max_y + max_offset + 0.1)
plt.subplots_adjust(top=0.98, bottom=0.08, left=0.35, right=0.95)
fig.savefig("../figure/fig_1_frequency.pdf", bbox_inches="tight", pad_inches=0.02, dpi=300)
